In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
with open('sherlock-holm.es_stories_plain-text_advs.txt','r',encoding='utf-8') as file:
    text = file.read()

In [3]:
## Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
print(total_words)

8200


In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,maxlen = max_sequence_len, padding = 'pre'))

In [7]:
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [8]:
# y= np.array(tf.keras.utils.to_categorical(y,num_classes = total_words))
from tensorflow.keras.utils import to_categorical
batch_size = 10000
epochs =100

In [9]:
## Neural Network model
model = Sequential()
model.add(Embedding(input_dim = total_words,output_dim=100,input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics =['accuracy'])
print(model.summary())

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
# for epoch in range(epochs):
#     for i in range(0, len(y), batch_size):
#         print(f"Epoch {epoch + 1}/{epochs}")
#         # Prepare the input and target batches
#         X_batch = X[i:i + batch_size]
#         y_batch = y[i:i + batch_size]
        
#         # One-hot encode the target batch
#         y_batch_one_hot = to_categorical(y_batch, num_classes=total_words)
        
#         # Train on the batch
#         model.train_on_batch(X_batch, y_batch_one_hot)
#     print(f"Epoch {epoch + 1} completed.")

# print("Training complete.")
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, total_words):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.total_words = total_words
    
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_X = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y_one_hot = to_categorical(batch_y, num_classes=self.total_words)
        return batch_X, batch_y_one_hot

# Initialize generator with smaller batch size
batch_size = 1000
data_generator = DataGenerator(X, y, batch_size, total_words)

# Train model using the data generator
model.fit(data_generator, epochs=100, verbose=1)


Epoch 1/100


C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 76s 706ms/step - accuracy: 0.0425 - loss: 7.6282
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 758ms/step - accuracy: 0.0551 - loss: 6.3746
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 71s 728ms/step - accuracy: 0.0557 - loss: 6.2300
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 69s 713ms/step - accuracy: 0.0561 - loss: 6.1306
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 68s 705ms/step - accuracy: 0.0671 - loss: 6.0500
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 82s 704ms/step - accuracy: 0.0730 - loss: 5.9497
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 69s 709ms/step - accuracy: 0.0792 - loss: 5.8485
Epoch 8/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 69s 709ms/step - accuracy: 0.0966 - loss: 5.7162
Epoch 9/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 68s 703ms/step - accuracy: 0.1079 - loss: 5.6087
Epoch 10/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 68s 703ms/step - accuracy: 0.1163 - loss: 5.5337
Epoch 11/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 68s 703ms/step - accuracy: 0.1250 - loss: 5.4483
Epoch 12/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 68